# This is going to be epic ... sit back, relax and enjoy !

copy from https://www.kaggle.com/raddar/target-true-meaning-revealed/

1. Following my previous kernel (https://www.kaggle.com/raddar/towards-de-anonymizing-the-data-some-insights) I was able to reveal that target is transformed using **log function**, and the raw target can be reversed with 2**target transformation. 

2. Moreover, I speculated that the true target is **a ratio of product_sum**.

3. This kernel is all about **explaining**, what kind of ratio we are working with, and why the problem is so hard! And this is going to blow your mind!

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.10f}'.format)

train = pd.read_csv('./data/train.csv')
historical_transactions = pd.read_csv('./data/historical_transactions.csv')
new_merchant_transactions = pd.read_csv('./data/new_merchant_transactions.csv')

# To a meaningfull form: purchase_amount & target 

Let's apply transformations as in previous kernel:

In [2]:
def shift_round(col):
    # shift_round: reverse normalization ?
    # where are the two numbers come from ?
    return np.round(col / 0.00150265118 + 497.06, 2)

new_merchant_transactions['purchase_amount_new'] = shift_round(new_merchant_transactions['purchase_amount'])
historical_transactions['purchase_amount_new'] = shift_round(historical_transactions['purchase_amount'])
train['target_raw'] = 2**train['target']

In [3]:
historical_transactions[["purchase_amount", "purchase_amount_new"]].head()

,purchase_amount,purchase_amount_new
0,-0.7033309100,29.0000000000
1,-0.7331284800,9.1700000000
2,-0.7203860000,17.6500000000
3,-0.7353524100,7.6900000000
4,-0.7228653800,16.0000000000


In [4]:
new_merchant_transactions[["purchase_amount", "purchase_amount_new"]].head()

,purchase_amount,purchase_amount_new
0,-0.5575737500,126.0000000000
1,-0.5695799300,118.0100000000
2,-0.5510372100,130.3500000000
3,-0.6719255000,49.9000000000
4,-0.6599042900,57.9000000000


# Find the simple case (stage 2)

Now in order to prove that target is a meaningful ratio, 

1. I was looking for some easy cases, such as **card_id with only one merchant_id**, with no transactions in new_merchant_transaction_table. 

2. This helps to **isolate the problem** and analyze it more thoroughly.

3. With some **manual exploration**, I found that merchant_id=M_ID_fc7d7969c3 is a perfect candidate to work with.

In [21]:
# find two merchant
merchant_counts = (historical_transactions.groupby(['card_id'])['merchant_id']
                                          .nunique().reset_index(name='merchant_n'))
one_merchant = merchant_counts[merchant_counts['merchant_n']==2].reset_index(drop=True)
print("one_merchant.shape:", one_merchant.shape)
print(one_merchant.head())

# no transactions in new_merchant_transactions
dat = historical_transactions.loc[historical_transactions['card_id'].isin(one_merchant['card_id'])]
print("historical one merchant records", dat.shape)
print(dat.head())
dat = dat.loc[~dat['card_id'].isin(new_merchant_transactions['card_id'])]
print("not in new_merchant_transactions")
print(dat.shape)
print(dat.head())
print("number of unique merchant ids", dat.merchant_id.nunique())

one_merchant.shape: (2367, 2)
           card_id  merchant_n
0  C_ID_00167ca6d8           2
1  C_ID_0024770552           2
2  C_ID_00251f994c           2
3  C_ID_0028e15a78           2
4  C_ID_002c77f4ae           2
historical one merchant records (39116, 15)
       authorized_flag          card_id  city_id category_1  installments  \
187933               Y  C_ID_3f0a4b6f48       69          N             1   
187934               Y  C_ID_3f0a4b6f48       -1          Y             1   
187935               Y  C_ID_3f0a4b6f48       -1          Y             1   
187936               Y  C_ID_3f0a4b6f48       -1          Y             1   
187937               Y  C_ID_3f0a4b6f48       -1          Y             1   

       category_3  merchant_category_id      merchant_id  month_lag  \
187933          B                   661  M_ID_9e84cda3b1         -8   
187934          B                   661  M_ID_fc7d7969c3          0   
187935          B                   661  M_ID_fc7d7969c3        

In [24]:
part = dat.loc[dat.merchant_id=='M_ID_fc7d7969c3'].reset_index(drop=True)
print(part.shape)

dat = dat.loc[dat.card_id.isin(part.card_id.unique())].reset_index(drop=True)

print("number of unique card ids", dat.card_id.nunique())
print(dat.head())

(2388, 15)
number of unique card ids 277
  authorized_flag          card_id  city_id category_1  installments  \
0               Y  C_ID_3f0a4b6f48       69          N             1   
1               Y  C_ID_3f0a4b6f48       -1          Y             1   
2               Y  C_ID_3f0a4b6f48       -1          Y             1   
3               Y  C_ID_3f0a4b6f48       -1          Y             1   
4               Y  C_ID_3f0a4b6f48       -1          Y             1   

  category_3  merchant_category_id      merchant_id  month_lag  \
0          B                   661  M_ID_9e84cda3b1         -8   
1          B                   661  M_ID_fc7d7969c3          0   
2          B                   661  M_ID_fc7d7969c3        -11   
3          B                   661  M_ID_fc7d7969c3         -9   
4          B                   661  M_ID_fc7d7969c3         -3   

   purchase_amount        purchase_date   category_2  state_id  subsector_id  \
0    -0.4961754200  2017-06-28 05:10:01 1.0000000

Let's take a look at a random card_id to inspect the transactional history:

In [29]:
historical_transactions[historical_transactions.card_id=='C_ID_3f0a4b6f48'].sort_values(["merchant_id", 'purchase_date'])[['card_id', 'merchant_id','purchase_date','purchase_amount_new']]

,card_id,merchant_id,purchase_date,purchase_amount_new
187951,C_ID_3f0a4b6f48,M_ID_9e84cda3b1,2017-01-28 05:25:37,124.9000000000
187943,C_ID_3f0a4b6f48,M_ID_9e84cda3b1,2017-02-28 01:05:23,159.9000000000
187938,C_ID_3f0a4b6f48,M_ID_9e84cda3b1,2017-03-28 04:58:05,159.9000000000
187953,C_ID_3f0a4b6f48,M_ID_9e84cda3b1,2017-05-28 09:35:44,161.8600000000
187933,C_ID_3f0a4b6f48,M_ID_9e84cda3b1,2017-06-28 05:10:01,166.8600000000
187941,C_ID_3f0a4b6f48,M_ID_9e84cda3b1,2017-07-28 02:43:51,168.8600000000
187940,C_ID_3f0a4b6f48,M_ID_9e84cda3b1,2017-08-28 02:19:30,168.8600000000
187949,C_ID_3f0a4b6f48,M_ID_9e84cda3b1,2017-09-28 06:16:35,168.8600000000
187944,C_ID_3f0a4b6f48,M_ID_fc7d7969c3,2017-01-15 08:35:51,22.9000000000
187946,C_ID_3f0a4b6f48,M_ID_fc7d7969c3,2017-02-15 08:27:12,22.9000000000


In [30]:
new_merchant_transactions[new_merchant_transactions.card_id=='C_ID_3f0a4b6f48'].sort_values('purchase_date')

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount_new


The payments are happeninig roughly on the same day of month (maybe recurring credit card payments). We also observe a purchase_amount increase from 29.9 to 37.9.

So why the merchant_id=M_ID_fc7d7969c3 is so special? It turns out it is subscribtion based merchant - most likely internet payments???The purchase_amount is distributed in a few very distinct categories:

In [31]:
historical_transactions.loc[historical_transactions.merchant_id=='M_ID_9e84cda3b1'].groupby('purchase_amount_new')['card_id'].count()

purchase_amount_new
0.0100000000        5
0.0200000000        1
0.0400000000        1
0.1000000000        1
0.2700000000        1
0.8400000000        2
0.9200000000        1
1.0000000000        6
1.0800000000        1
1.1000000000        1
1.2800000000        1
1.4800000000        1
2.0000000000        2
2.0300000000        2
2.1500000000        1
2.1700000000        1
2.2600000000        1
2.8000000000        1
3.0000000000       10
3.2800000000        1
3.2900000000        1
3.3100000000        1
3.4500000000        1
3.5900000000        1
3.7500000000        1
3.7900000000        1
4.0000000000        2
4.3600000000        2
4.8900000000        1
5.0000000000        1
                   ..
1079.5000000000     1
1087.7100000000     1
1113.0800000000     2
1114.2500000000     2
1125.7900000000     1
1127.9600000000     1
1185.2600000000     1
1188.3300000000     1
1193.5600000000     1
1211.2800000000     1
1228.8600000000     1
1231.2600000000     1
1259.8300000000     1
1261.8800000

In [8]:
historical_transactions.loc[historical_transactions.merchant_id=='M_ID_fc7d7969c3'].groupby('purchase_amount_new')['card_id'].count()

purchase_amount_new
1.0000000000     45054
9.5000000000         2
13.9500000000        1
19.9000000000    19709
22.9000000000    31680
27.9000000000    33017
29.9000000000    24097
37.9000000000    23480
Name: card_id, dtype: int64

This is a very useful information - remember - the hypothesis was that target is based on some kind of ratios, and the purchase_amount ratio seems like a very reasonable candidate.

At this point let's summarise what we know:

1. merchant is selling subscribtion based products
2. card_id is automatically charged monthly
3. the price for the subscribtion can change (most likely due to upselling)
4. there are 5 products with prices of 19.90, 22.90, 27.90, 29.90 and 37.90; the 1.00 is probably subscribtion activation fee

Now let's try to make some correlations with train target:

In [9]:
print(train.shape)
print(train.card_id.nunique())

(201917, 7)
201917


In [32]:
dat = dat.merge(train, on = 'card_id')

In [33]:
dat.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,category_2,state_id,subsector_id,purchase_amount_new,first_active_month,feature_1,feature_2,feature_3,target,target_raw
0,Y,C_ID_3f0a4b6f48,69,N,1,B,661,M_ID_9e84cda3b1,-8,-0.4961754200,...,1.0000000000,9,8,166.8600000000,2015-09,1,2,0,0.0000000000,1.0000000000
1,Y,C_ID_3f0a4b6f48,-1,Y,1,B,661,M_ID_fc7d7969c3,0,-0.7049838300,...,nan,-1,8,27.9000000000,2015-09,1,2,0,0.0000000000,1.0000000000
2,Y,C_ID_3f0a4b6f48,-1,Y,1,B,661,M_ID_fc7d7969c3,-11,-0.7124970800,...,nan,-1,8,22.9000000000,2015-09,1,2,0,0.0000000000,1.0000000000
3,Y,C_ID_3f0a4b6f48,-1,Y,1,B,661,M_ID_fc7d7969c3,-9,-0.7124970800,...,nan,-1,8,22.9000000000,2015-09,1,2,0,0.0000000000,1.0000000000
4,Y,C_ID_3f0a4b6f48,-1,Y,1,B,661,M_ID_fc7d7969c3,-3,-0.7049838300,...,nan,-1,8,27.9000000000,2015-09,1,2,0,0.0000000000,1.0000000000


In [34]:
dat.groupby('target_raw')['card_id'].count().reset_index(name='n')

,target_raw,n
0,0.0000000001,24
1,0.1065292098,6
2,0.1592102263,120
3,0.2577319592,8
4,0.2689156625,10
5,0.2823500398,4
6,0.3143921542,10
7,0.5000000000,41
8,0.5250659622,14
9,0.5348952102,14


In [12]:
dat.groupby('target_raw')['card_id'].count().reset_index(name='n')

,target_raw,n
0,0.0000000001,23
1,0.0218079291,13
2,0.5000000000,32
3,0.6666666670,2
4,0.7132616494,6
5,0.7991967880,5
6,0.9998680889,3
7,1.0000000000,2294
8,1.0001319285,17
9,1.0001792085,15


It seems most of the target values for this specific card_id group is concentrated in **target_raw=1**. 

This is great news, because we were able to somehow **cluster this specific group of cards just by some ad hoc rules** (single merchant, specific merchant, no new merchant transactions).

Let's take a look at card_ids, which target_raw is equal to 1:

In [35]:
(dat.loc[dat['target_raw']==1,['card_id','purchase_date','purchase_amount_new','target_raw']]
    .sort_values(['card_id','purchase_date'])
    .head())

,card_id,purchase_date,purchase_amount_new,target_raw
599,C_ID_026cb25282,2017-10-29 16:43:45,1.0000000000,1.0000000000
591,C_ID_026cb25282,2017-10-29 16:44:06,1.0000000000,1.0000000000
592,C_ID_026cb25282,2017-10-29 16:44:55,1.0000000000,1.0000000000
601,C_ID_026cb25282,2017-10-29 16:44:58,27.9000000000,1.0000000000
598,C_ID_026cb25282,2017-11-27 13:21:06,89.9000000000,1.0000000000


In [36]:
dat[dat.target_raw == 1].card_id.unique()[:10]

array(['C_ID_3f0a4b6f48', 'C_ID_94d2dfafec', 'C_ID_4cf5db0735',
       'C_ID_ea7f034e3e', 'C_ID_1010ee7d0e', 'C_ID_026cb25282',
       'C_ID_4de715fff3', 'C_ID_99722dd1ef', 'C_ID_2d77b7d713',
       'C_ID_4fbc5d0607'], dtype=object)

In [38]:
for card_id in dat[dat.target_raw == 1].card_id.unique()[:10]:
    print(card_id)
    records = dat[dat.card_id == card_id].sort_values(['merchant_id', 'purchase_date'])[['card_id', 'merchant_id' ,'purchase_date','purchase_amount_new']]
    print(records)

C_ID_3f0a4b6f48
            card_id      merchant_id        purchase_date  purchase_amount_new
18  C_ID_3f0a4b6f48  M_ID_9e84cda3b1  2017-01-28 05:25:37       124.9000000000
10  C_ID_3f0a4b6f48  M_ID_9e84cda3b1  2017-02-28 01:05:23       159.9000000000
5   C_ID_3f0a4b6f48  M_ID_9e84cda3b1  2017-03-28 04:58:05       159.9000000000
20  C_ID_3f0a4b6f48  M_ID_9e84cda3b1  2017-05-28 09:35:44       161.8600000000
0   C_ID_3f0a4b6f48  M_ID_9e84cda3b1  2017-06-28 05:10:01       166.8600000000
8   C_ID_3f0a4b6f48  M_ID_9e84cda3b1  2017-07-28 02:43:51       168.8600000000
7   C_ID_3f0a4b6f48  M_ID_9e84cda3b1  2017-08-28 02:19:30       168.8600000000
16  C_ID_3f0a4b6f48  M_ID_9e84cda3b1  2017-09-28 06:16:35       168.8600000000
11  C_ID_3f0a4b6f48  M_ID_fc7d7969c3  2017-01-15 08:35:51        22.9000000000
13  C_ID_3f0a4b6f48  M_ID_fc7d7969c3  2017-02-15 08:27:12        22.9000000000
2   C_ID_3f0a4b6f48  M_ID_fc7d7969c3  2017-03-15 09:30:03        22.9000000000
15  C_ID_3f0a4b6f48  M_ID_fc7d7969c3

In [40]:
for card_id in dat[dat.target_raw != 1].card_id.unique()[:10]:
    print(card_id)
    records = dat[dat.card_id == card_id].sort_values(['merchant_id', 'purchase_date'])[['card_id', 'merchant_id' ,'purchase_date','purchase_amount_new', 'target_raw']]
    print(records)

C_ID_8da6ff1ae7
            card_id      merchant_id        purchase_date  \
22  C_ID_8da6ff1ae7  M_ID_57df19bf28  2018-01-17 10:53:47   
24  C_ID_8da6ff1ae7  M_ID_57df19bf28  2018-02-12 05:30:45   
26  C_ID_8da6ff1ae7  M_ID_fc7d7969c3  2017-10-06 03:38:35   
25  C_ID_8da6ff1ae7  M_ID_fc7d7969c3  2017-11-05 09:10:13   
27  C_ID_8da6ff1ae7  M_ID_fc7d7969c3  2017-12-05 08:40:53   
23  C_ID_8da6ff1ae7  M_ID_fc7d7969c3  2018-01-05 11:05:40   
28  C_ID_8da6ff1ae7  M_ID_fc7d7969c3  2018-02-05 07:49:23   

    purchase_amount_new   target_raw  
22        39.8900000000 2.1741407336  
24        39.8900000000 2.1741407336  
26        19.9000000000 2.1741407336  
25        27.9000000000 2.1741407336  
27        27.9000000000 2.1741407336  
23        27.9000000000 2.1741407336  
28        27.9000000000 2.1741407336  
C_ID_37c961ab33
             card_id      merchant_id        purchase_date  \
182  C_ID_37c961ab33  M_ID_9139332ccc  2017-01-04 13:15:38   
268  C_ID_37c961ab33  M_ID_9139332ccc  2017

In [46]:
grp = dat.groupby(["card_id", "merchant_id"]).purchase_amount_new.std().reset_index(name="cm_std").groupby("merchant_id")

merchant_mean_std = pd.DataFrame()
merchant_mean_std["card_cnt"] = grp.size()
merchant_mean_std["avg_cm_std"] = grp.cm_std.mean()

print(merchant_mean_std.shape)
merchant_mean_std.head()

(86, 2)


,card_cnt,avg_cm_std
merchant_id,,
M_ID_00a6ca8a8a,5,nan
M_ID_03de385551,1,0.0000000000
M_ID_076cd8a829,1,nan
M_ID_09d7a200d1,1,nan
M_ID_0da765990b,1,0.0000000000


In [48]:
merchant_mean_std.sort_values("card_cnt").tail()

,card_cnt,avg_cm_std
merchant_id,,
M_ID_57df19bf28,7,5.0587995899
M_ID_6f274b9340,9,40.6159460357
M_ID_86be58d7e0,14,2.0425675612
M_ID_9139332ccc,19,156.8595157534
M_ID_fc7d7969c3,164,7.6663933099


# A ratio

1. So there is some **fluctuation** in purchase_amount, therefore it is still unclear why the target_raw==1.

2. However, the most interesting part lies not in target_raw=1, but actually in **other float numbers**!

Let's take a step back first and revisit the purchase_amount values we extracted earlier, and let's calculate possible ratios based on these values:

In [16]:
list(historical_transactions.loc[historical_transactions.merchant_id=='M_ID_fc7d7969c3'].groupby('purchase_amount_new')['card_id'].count().index)

[1.0, 9.5, 13.95, 19.9, 22.9, 27.9, 29.9, 37.9]

In [17]:
prices = [19.90, 22.90, 27.90, 29.90, 37.90]
prices = [59.65, 60.18, 60.46, 60.84, 61.03, 61.34, 61.47, 61.99]
sorted({ i/j for j in prices for i in prices})

[0.9622519761251814,
 0.9703920611680494,
 0.9708017422164865,
 0.9724486468862079,
 0.9753185997741571,
 0.9773881697525807,
 0.9790141532454856,
 0.9804404996712688,
 0.9810890120639061,
 0.9814486207452815,
 0.9835692207580934,
 0.9845136312308437,
 0.9856537332898597,
 0.9860724233983287,
 0.9866027125372147,
 0.9891518737672583,
 0.9895144378125504,
 0.9897510980966325,
 0.9906603309847616,
 0.9911930874044533,
 0.9916115502500403,
 0.9918487120965113,
 0.9928420367659021,
 0.9937541091387245,
 0.9949462014998369,
 0.9953688389017532,
 0.9968867769949206,
 0.9978851472262893,
 1.0,
 1.002119334854907,
 1.0031229454306376,
 1.0046527085410435,
 1.0050794691135507,
 1.0062851472047636,
 1.0072095690643945,
 1.0082182774490467,
 1.008459411094843,
 1.0088851634534786,
 1.0094277208071452,
 1.0103550295857988,
 1.0105966742745354,
 1.0109670987038883,
 1.0135792120704108,
 1.0141242937853108,
 1.014555077737347,
 1.01572996886777,
 1.0167052596758186,
 1.0189020381328073,
 1.019275506

These numbers are not telling anything at the moment.

But there is the catch - let's revisit the previous table, were we calculated unique target_raw values (compare the list above with the table below).

Hint: if you looked closely you may find that some of the list values overlap!

In [18]:
def min_offset(x):
    return np.abs((np.array(sorted({ i/j for j in prices for i in prices})) - x)).min()

part = dat.groupby('target_raw')['card_id'].count().reset_index(name='n')
part["min_offset"] = part.target_raw.apply(min_offset)
part.sort_values("min_offset").head()

,target_raw,n,min_offset
7,1.0000000000,2294,0.0000000000
6,0.9998680889,3,0.0001319111
8,1.0001319285,17,0.0001319285
9,1.0001792085,15,0.0001792085
10,1.0941048039,18,0.0548759690


You will find the values like 0.713261, 1.21834, 1.35842 appearing in both objects!

This is pretty amazing find and at this point I am 100% sure that the target is a ratio of change in purchase_amount.

However, there is still a question to ask - how exactly this ratio is calculated?

Let's take this card_id for example:

In [19]:
dat[dat.card_id=='C_ID_2c8d99614f'].sort_values('purchase_date')[['card_id','purchase_date','purchase_amount_new','target_raw']]

,card_id,purchase_date,purchase_amount_new,target_raw
2245,C_ID_2c8d99614f,2017-06-11 23:59:54,1.0000000000,1.2183406082
2241,C_ID_2c8d99614f,2017-06-11 23:59:59,22.9000000000,1.2183406082
2250,C_ID_2c8d99614f,2017-07-11 08:50:47,22.9000000000,1.2183406082
2239,C_ID_2c8d99614f,2017-08-11 06:41:07,22.9000000000,1.2183406082
2248,C_ID_2c8d99614f,2017-09-11 18:46:07,22.9000000000,1.2183406082
2247,C_ID_2c8d99614f,2017-10-11 09:28:10,22.9000000000,1.2183406082
2246,C_ID_2c8d99614f,2017-11-11 10:46:12,22.9000000000,1.2183406082
2240,C_ID_2c8d99614f,2017-12-11 07:36:29,22.9000000000,1.2183406082
2243,C_ID_2c8d99614f,2018-01-11 09:54:39,22.9000000000,1.2183406082
2244,C_ID_2c8d99614f,2018-01-15 16:14:59,22.9000000000,1.2183406082


If we excluded the subscribtion activation fees (1.00), you would expect the target_raw to be 1 (as purchase_amount is constant...)

However the target_raw is 1.2183406082, which in fact is equivalent to 27.90/22.90.

Amazing! Now we definetly know that the purchase_amount for this card_id has changed, and it is very likely happened in the future (month_lag = 1 or month_lag = 2). This information we do not observe in new_merchant_transactions table, because organizers have excluded that on purpose in data preparation stage - if we would have this information, we could easily track down how target has been calculated, and 0.000 RMSE score would be possible.

What does this all mean for target calculation?

The answer is simple:

> target is equal to the ratio of money spent in the future divided by money spent in the past!

I guess this transfers to all merchants and to all cards in the dataset, but the calculations are a bit harder, as more merchants are involved.

# Bonus


Now as we know that we are working with future/history ratios, the outlier value (-33.xxxxx) is actually the meaning of a user not spending a single cent in the future on the historical merchants - mystery solved!

If i was an organizer, I would have made it to be ~ -10 or so, as RMSE is so sensitive regarding this outler'ish value...

Sadly, this also means that these outliers cannot be predicted with the given historical information...

Knowing all this I can say for sure that the data we have is in fact real data and not simulated one! In case I am wrong on this one - well done Elo - you did a marvelous job in simulation part (would be hard to make a clean generator like that!)

# Things that are still not clear...

Now as we definetly know that target is a ratio of future and past transaction amounts, there are still some unaswered questions, mainly:

which months are taken into calculation? month_lag=1 / month_lag=0, month_lag=2 / month_lag=0, month_lag=1 / min(month_lag=0, month_lag=-1), etc. All these are all valid options...
does new_merchant_transactions have any influence for the target?

# What's next?

So what you can do with this kind of information. 

1. Firstly, the rate of purchase_amount monthly change for (card_id, merchant_id) tuple should be the key features in your models, i.e. average change ratio of (5->4, 4->3, 3->2, 2->1, 1->0); 

2. Or features such as sum(future_purchase_amount)/sum(purchase_amount_lag_-1), etc.

3. This also allows building new intermediate models, like predicting the historic merchant performance in month_lag=1 and month_lag=2 and stacking its predictions to your main model.

These are the most impactful things I can think of right now, but there could always be more!